# Dataset Import

In [1]:
import pandas as pd

In [20]:
fr_train = pd.read_csv("fr_train_outlier.csv", index_col=0)
fr_train

,ID,DAY_ID,FR_CONSUMPTION,FR_DE_EXCHANGE,FR_NET_EXPORT,FR_NET_IMPORT,FR_GAS,FR_COAL,FR_HYDRO,FR_NUCLEAR,FR_SOLAR,FR_WINDPOW,FR_RESIDUAL_LOAD,FR_RAIN,FR_WIND,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,TARGET
1,1179,1,1.222131,-0.331356,0.778627,-0.778627,1.991028,-0.786509,0.709616,1.381575,0.485975,-0.172140,1.214288,-0.497520,-1.465608,0.231602,1.480313,0.931562,0.822047,-0.063369
2,1327,2,-0.667390,1.102015,0.256736,-0.256736,0.458302,-0.766904,-0.930172,-0.379230,1.032412,-0.844350,-0.540642,-0.372156,-0.926064,0.641235,1.802550,1.140920,0.900434,2.575976
3,2016,3,-0.834564,1.051716,-0.612133,0.612133,0.069297,-0.718729,-0.383690,-1.579208,2.986527,-0.718643,-0.856321,-1.118297,0.488650,-0.951057,0.440121,-0.064550,-0.032756,0.068905
4,2047,5,-0.470371,-0.144615,-1.811403,1.811403,0.528273,-0.766063,-0.398178,-1.866010,3.425813,-0.640389,-0.552878,-0.790071,0.021868,1.459745,-0.117977,0.550433,0.781870,1.031308
5,1995,7,-0.625625,-0.002239,-0.745182,0.745182,0.727314,-0.778036,-0.739291,-1.934168,2.276123,-0.079343,-0.806379,-0.663419,1.367421,0.954384,-0.379980,0.518459,-0.034642,-0.118915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,1120,1205,-0.100235,-0.381612,0.106216,-0.106216,0.137448,0.578370,-0.191697,0.132869,0.127692,-0.389945,-0.019808,-0.435577,-0.667096,-0.481947,0.047390,0.562084,-0.954402,-0.028575
845,1721,1207,0.516789,-0.651648,-0.820640,0.820640,0.295393,-0.783923,-0.138441,-0.523101,1.539418,0.839589,0.236243,1.112782,-0.018442,-0.960526,0.099209,1.750872,0.646905,-0.729755
846,2039,1208,-0.709011,-0.427976,-1.678101,1.678101,0.179728,-0.758579,-0.380974,-1.903612,1.061313,-0.468104,-0.666261,2.661142,0.630211,-1.439105,0.291714,0.568479,0.395742,0.136028
848,1987,1212,-0.520506,0.270515,-0.789824,0.789824,0.625656,-0.776785,-0.906285,-1.434474,1.589641,-0.266687,-0.589767,-0.237434,1.123953,-0.308232,0.017778,0.072168,-0.160792,-0.425474


In [27]:
fr_train = fr_train.drop('DAY_ID', axis=1)
print(fr_train.columns)

Index(['ID', 'FR_CONSUMPTION', 'FR_DE_EXCHANGE', 'FR_NET_EXPORT',
       'FR_NET_IMPORT', 'FR_GAS', 'FR_COAL', 'FR_HYDRO', 'FR_NUCLEAR',
       'FR_SOLAR', 'FR_WINDPOW', 'FR_RESIDUAL_LOAD', 'FR_RAIN', 'FR_WIND',
       'FR_TEMP', 'GAS_RET', 'COAL_RET', 'CARBON_RET', 'TARGET'],
      dtype='object')


In [22]:
from sklearn.model_selection import train_test_split

fr_X = fr_train.drop(['TARGET', 'FR_NET_EXPORT'], axis=1)
fr_y = fr_train['TARGET']

fr_X_train, fr_X_valid, fr_y_train, fr_y_valid = train_test_split(fr_X, fr_y, test_size=0.2, random_state=42)

print("train dataset:", fr_X_train.shape, fr_y_train.shape)
print("validation dataset:", fr_X_valid.shape, fr_y_valid.shape)

train dataset: (564, 17) (564,)
validation dataset: (141, 17) (141,)


In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import spearmanr
from sklearn.metrics import make_scorer

# scorer
def spearman_corr(y_true, y_pred):
    return spearmanr(y_true, y_pred).correlation

spearman_scorer = make_scorer(spearman_corr)

param_grid = {
    'n_estimators': [50, 100, 200],  # num of tree
    'max_features': ['auto', 'sqrt', 'log2'], 
    'max_depth': [None, 10, 20, 30],  # max depth of tree
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [0, 1, 2, 4], 
    'bootstrap': [True, False] 
}

rf = RandomForestRegressor(random_state=42)
print(rf)

# GridSearch
crf = GridSearchCV(rf, param_grid=param_grid, cv=5, scoring=spearman_scorer, n_jobs=-1)
crf.fit(fr_X_train, fr_y_train)

print("Best Parameters:", crf.best_params_)
print("Best Score:", crf.best_score_)


RandomForestRegressor(random_state=42)
Best Parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Best Score: 0.17114674963776774


/Users/wangjiuduo/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
2160 fits failed out of a total of 4320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
531 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/wangjiuduo/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/wangjiuduo/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/Users/wangjiuduo/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_con

In [25]:
best_model = crf.best_estimator_
validation_score = spearman_corr(fr_y_valid, best_model.predict(fr_X_valid))

print("Validation Score with Best Parameters:", validation_score)

Validation Score with Best Parameters: 0.24430840361316836
